In [1]:
from collections import deque
import heapq, math

In [2]:
def normalize_state(state, precision=6):
    """Round state values to avoid floating-point drift."""
    return tuple(round(x, precision) for x in state)

In [3]:
def reached_goal(current, goal, tol=1e-6):
    """Check if current state matches the target within tolerance."""
    return all(abs(current[i] - goal[i]) <= tol for i in range(3))

In [4]:
def expand_state(state, limits):
    """Generate all valid successor states by transferring liquid."""
    successors = []
    for src_idx in range(3):
        for dst_idx in range(3):
            if src_idx == dst_idx:
                continue
            src_val, dst_val = state[src_idx], state[dst_idx]
            cap_src, cap_dst = limits[src_idx], limits[dst_idx]

            # Source must retain 20% of its capacity
            min_reserve = 0.2 * cap_src
            transferable = src_val - min_reserve
            space = cap_dst - dst_val
            move_amt = min(transferable, space)

            if move_amt > 1e-9:
                new_state = list(state)
                new_state[src_idx] -= move_amt
                new_state[dst_idx] += move_amt
                successors.append((
                    normalize_state(tuple(new_state)),
                    f"Open valve ({src_idx+1}->{dst_idx+1})",
                    move_amt
                ))
    return successors

In [5]:
# --------- BFS ----------
# Breadth-First Search (BFS):
# - Explores states level by level using a queue (FIFO).
# - Always finds the solution with the minimum number of valve operations.
# - Tracks visited states to avoid loops and reconstructs path from parent mapping.


def breadth_first_search(start, goal, limits):
    start = normalize_state(start)
    goal = normalize_state(goal)

    queue = deque([start])
    parent, action_taken = {start: None}, {}

    while queue:
        cur = queue.popleft()
        if reached_goal(cur, goal):
            # Reconstruct path
            steps = []
            while parent[cur] is not None:
                steps.append(action_taken[cur])
                cur = parent[cur]
            return list(reversed(steps))

        for nxt, act, _ in expand_state(cur, limits):
            if nxt not in parent:
                parent[nxt] = cur
                action_taken[nxt] = act
                queue.append(nxt)
    return None

In [6]:
# --------- DFS ----------
# - Explores one path deeply before backtracking, using recursion or stack (LIFO).
# - May find a solution quickly but not guaranteed to be optimal.
# - Tracks visited states to prevent infinite loops.

def depth_first_search(start, goal, limits, max_depth=1000):
    start = normalize_state(start)
    goal = normalize_state(goal)

    stack = [(start, 0)]
    parent, action_taken = {start: None}, {}
    depth_seen = {start: 0}

    while stack:
        cur, depth = stack.pop()
        if reached_goal(cur, goal):
            steps = []
            while parent[cur] is not None:
                steps.append(action_taken[cur])
                cur = parent[cur]
            return list(reversed(steps))

        if depth >= max_depth:
            continue

        for nxt, act, _ in expand_state(cur, limits):
            prev_depth = depth_seen.get(nxt, None)
            if prev_depth is None or depth + 1 < prev_depth:
                parent[nxt] = cur
                action_taken[nxt] = act
                depth_seen[nxt] = depth + 1
                stack.append((nxt, depth + 1))
    return None

In [7]:
# --------- A* ----------
# - Uses a priority queue to expand states with lowest cost f(n) = g(n) + h(n).
# - g(n) = cost so far (number of valve operations).
# - h(n) = heuristic estimate of steps to reach target .
# - Guarantees an optimal solution while exploring fewer states than BFS.

def admissible_heuristic(state, goal, limits):
    """Estimate moves needed: based on max possible transfer volume."""
    total_diff = sum(abs(state[i] - goal[i]) for i in range(3))
    required_transfer = total_diff / 2.0
    max_flow = max(0.8 * c for c in limits)
    return 0 if max_flow <= 0 else math.ceil(required_transfer / max_flow)

def a_star_search(start, goal, limits):
    start = normalize_state(start)
    goal = normalize_state(goal)

    g_costs = {start: 0}
    parent, action_taken = {start: None}, {}

    init_h = admissible_heuristic(start, goal, limits)
    frontier = [(init_h, 0, start)]
    explored = {}

    while frontier:
        f, g, cur = heapq.heappop(frontier)
        if cur in explored and explored[cur] <= g:
            continue
        explored[cur] = g

        if reached_goal(cur, goal):
            steps = []
            while parent[cur] is not None:
                steps.append(action_taken[cur])
                cur = parent[cur]
            return list(reversed(steps))

        for nxt, act, _ in expand_state(cur, limits):
            new_g = g + 1
            if nxt in g_costs and new_g >= g_costs[nxt]:
                continue
            g_costs[nxt] = new_g
            parent[nxt] = cur
            action_taken[nxt] = act
            h = admissible_heuristic(nxt, goal, limits)
            heapq.heappush(frontier, (new_g + h, new_g, nxt))
    return None

In [9]:

# --------- Display ----------
def show_solution(alg_name, seq, start, limits):
    print(f"\n=== {alg_name} solution ===")
    if seq is None:
        print("No solution found.")
        return

    cur_state = list(start)
    print(f"Start state: {cur_state}")

    for step in seq:
        src, dst = map(int, step.replace("Open valve (","").replace(")","").split("->"))
        src -= 1; dst -= 1

        src_val, dst_val = cur_state[src], cur_state[dst]
        cap_src, cap_dst = limits[src], limits[dst]

        min_reserve = 0.2 * cap_src
        transferable = src_val - min_reserve
        space = cap_dst - dst_val
        moved = min(transferable, space)

        cur_state[src] -= moved
        cur_state[dst] += moved
        print(f"{step} -> {tuple(round(x,3) for x in cur_state)}")

    print(f"Number of valve operations = {len(seq)}")


# --------- Main Function----------
if __name__ == "__main__":
    C1, C2, C3 = map(float, input().split())
    I1, I2, I3 = map(float, input().split())
    T1, T2, T3 = map(float, input().split())

    capacities = (C1, C2, C3)
    start_state = (I1, I2, I3)
    goal_state = (T1, T2, T3)

    bfs_sol = breadth_first_search(start_state, goal_state, capacities)
    dfs_sol = depth_first_search(start_state, goal_state, capacities)
    astar_sol = a_star_search(start_state, goal_state, capacities)

    show_solution("BFS", bfs_sol, start_state, capacities)
    show_solution("DFS", dfs_sol, start_state, capacities)
    show_solution("A*", astar_sol, start_state, capacities)


8.0 5.0 3.0
8.0 0.0 0.0
2.4 5.0 0.6

=== BFS solution ===
Start state: [8.0, 0.0, 0.0]
Open valve (1->2) -> (3.0, 5.0, 0.0)
Open valve (1->3) -> (1.6, 5.0, 1.4)
Open valve (3->1) -> (2.4, 5.0, 0.6)
Number of valve operations = 3

=== DFS solution ===
Start state: [8.0, 0.0, 0.0]
Open valve (1->3) -> (5.0, 0.0, 3.0)
Open valve (3->2) -> (5.0, 2.4, 0.6)
Open valve (1->2) -> (2.4, 5.0, 0.6)
Number of valve operations = 3

=== A* solution ===
Start state: [8.0, 0.0, 0.0]
Open valve (1->2) -> (3.0, 5.0, 0.0)
Open valve (1->3) -> (1.6, 5.0, 1.4)
Open valve (3->1) -> (2.4, 5.0, 0.6)
Number of valve operations = 3
